Import required libraries for training

In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import keras
import os
from pathlib import Path
import shutil

Prepare created dataset

In [6]:
raw_dataset = pd.read_csv("dataset/data.csv")
size = raw_dataset["size"][0]
# path,kana,font,size,antialias
labels_map = json.load(open("dataset/mapping.json"))


def load_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=1)
    image = tf.image.convert_image_dtype(image, tf.float32)
    return image


def load_image_label(path, label):
    return load_image(path), label


def load_label(label):
    return labels_map[label]


def create_dataset():
    dataset = tf.data.Dataset.from_tensor_slices(
        (raw_dataset["path"], raw_dataset["kana"].map(load_label))
    )
    dataset = dataset.map(load_image_label)
    return dataset


def create_model():
    model = keras.models.Sequential(
        [
            keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(size, size, 1)),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation="relu"),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Conv2D(64, (3, 3), activation="relu"),
            keras.layers.Flatten(),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(len(labels_map), activation="softmax"),
        ]
    )
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


model = create_model()
dataset = create_dataset()

# dataset = dataset.shuffle(len(raw_dataset), seed=42)
images = np.array([image for image, label in dataset])
labels = np.array([label for image, label in dataset])
# train_size = int(len(raw_dataset) * 0.8)
# train_dataset = dataset.take(train_size)
# train_images = np.array([image for image, label in train_dataset])
# train_labels = np.array([label for image, label in train_dataset])
# test_dataset = dataset.skip(train_size)
# test_images = np.array([image for image, label in test_dataset])
# test_labels = np.array([label for image, label in test_dataset])

/home/fufsob/vuz/tpr/venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-15 12:29:36.987630: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-05-15 12:29:56.210388: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Train the model

In [14]:
print(images.shape)
print(labels)

(95658, 64, 64, 1)
[  0   1   2 ... 146 147 148]


In [7]:
np.random.seed(420)
output_dir = Path("output")
if output_dir.exists():
    shutil.rmtree(output_dir, ignore_errors=True)
output_dir.mkdir(exist_ok=True)
model.fit(
    images,
    labels,
    epochs=40,
    callbacks=[
        keras.callbacks.TensorBoard(log_dir='output/logs'),
        keras.callbacks.ModelCheckpoint('output/model_{epoch}.weights.h5', save_weights_only=True),
        keras.callbacks.ModelCheckpoint('output/model.keras', save_best_only=True),
        keras.callbacks.EarlyStopping("val_loss", patience=3),
    ],
    validation_freq=1,
    validation_split=0.2,
    shuffle=True,
)


Epoch 1/40


2024-05-15 12:29:57.631642: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1253801984 exceeds 10% of free system memory.


2392/2392 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6822 - loss: 1.3907

2024-05-15 12:31:40.603185: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 313458688 exceeds 10% of free system memory.


2392/2392 ━━━━━━━━━━━━━━━━━━━━ 107s 44ms/step - accuracy: 0.6822 - loss: 1.3904 - val_accuracy: 0.8941 - val_loss: 0.3723
Epoch 2/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 103s 43ms/step - accuracy: 0.9666 - loss: 0.0876 - val_accuracy: 0.8988 - val_loss: 0.4073
Epoch 3/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 112s 47ms/step - accuracy: 0.9747 - loss: 0.0593 - val_accuracy: 0.9194 - val_loss: 0.3022
Epoch 4/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 112s 47ms/step - accuracy: 0.9804 - loss: 0.0459 - val_accuracy: 0.9192 - val_loss: 0.3183
Epoch 5/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 100s 42ms/step - accuracy: 0.9825 - loss: 0.0396 - val_accuracy: 0.9234 - val_loss: 0.3177
Epoch 6/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 99s 41ms/step - accuracy: 0.9856 - loss: 0.0335 - val_accuracy: 0.9257 - val_loss: 0.2862
Epoch 7/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 98s 41ms/step - accuracy: 0.9867 - loss: 0.0305 - val_accuracy: 0.9200 - val_loss: 0.3698
Epoch 8/40
2392/2392 ━━━━━━━━━━━━━━━━━━━━ 98s 41ms/step - accuracy: 0.9876 - loss:

Predict the output

In [9]:
model: keras.models.Sequential = keras.models.load_model('output/model.keras')
model.evaluate(images, labels)

2024-05-15 14:18:48.438210: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1567260672 exceeds 10% of free system memory.


2990/2990 ━━━━━━━━━━━━━━━━━━━━ 29s 9ms/step - accuracy: 0.9850 - loss: 0.0358


[0.07927699387073517, 0.9752869606018066]